In [2]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import gemini_analyze_fuction
import gemini_comment_fuction
import gemini_setup_function
import gemini_summary_function
import get_and_crawl_data
import plotly_and_upload

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(gemini_analyze_fuction)
importlib.reload(gemini_comment_fuction)
importlib.reload(gemini_setup_function)
importlib.reload(gemini_summary_function)
importlib.reload(get_and_crawl_data)
importlib.reload(plotly_and_upload)

from import_default import *
from import_database import *
from import_other import *
from gemini_analyze_fuction import *
from gemini_comment_fuction import *
from gemini_setup_function import *
from gemini_summary_function import *
from get_and_crawl_data import *
from plotly_and_upload import *

#### Lấy dữ liệu Mongo

In [3]:
date_series = get_mongo_collection(ref_db, 'date_series')
time_series = get_mongo_collection(ref_db, 'time_series')
name_map = get_mongo_collection(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')

In [4]:
projection = {
    "_id": 0,
    "date": 1,
    "ticker": 1,
    "open": 1,
    "high": 1,
    "low": 1,
    "close": 1,
    "pct_change": 1,
    "diff": 1,
    "volume": 1,
    "option": 1,
    "SMA_20": 1,
    "SMA_60": 1,
    "RSI_14": 1,
    "week_open": 1,
    "month_prev_high": 1,
    "month_prev_low": 1,
    "month_open": 1,
    "quarter_prev_high": 1,
    "quarter_prev_low": 1,
    "quarter_open": 1,
    "year_open": 1,
    "MFIBO_0382": 1,
    "MFIBO_0500": 1,
    "MFIBO_0618": 1,
    "QFIBO_0382": 1,
    "QFIBO_0500": 1,
    "QFIBO_0618": 1,
	"YFIBO_0382": 1,
    "YFIBO_0500": 1,
    "YFIBO_0618": 1,
    'WPIVOT_P': 1,
}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

In [5]:
itd_index_df = get_mongo_collection(stock_db, "itd_index")
other_ticker_df = get_mongo_collection(stock_db, 'other_ticker', projection=projection)
nntd_index_df = get_mongo_collection(stock_db, 'nntd_index')
nntd_stock_df = get_mongo_collection(stock_db, 'nntd_stock')

#### Tạo các models cho gemini

In [6]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))

# Sắp xếp danh sách model theo thứ tự ổn định
# fast_model_list = select_fast_models(get_gemini_models())
# standard_model_list = select_standard_models(get_gemini_models())
fast_model_list = ['gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']
standard_model_list = ['gemini-2.5-flash', 'gemini-2.0-flash', 'gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']

# Tạo dictionary cho các model
fast_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in fast_model_list}
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

#### Dữ liệu thống kê giao dịch phiên hôm trước

- Tính toán bảng dữ liệu

In [7]:
itd_vn30f1m_df = itd_index_df[itd_index_df['ticker'] == 'VN30F1M'].reset_index(drop=True)
eod_vn30f1m_df = full_index_df[full_index_df['ticker'] == 'VN30F1M'].reset_index(drop=True)
eod_vn30_df = full_index_df[full_index_df['ticker'] == 'VN30'].reset_index(drop=True)

vn30f1m_close_prev = eod_vn30f1m_df[eod_vn30f1m_df['date'] == itd_vn30f1m_df['date'].iloc[-1].normalize()]['close'].item()
vn30_close_prev = eod_vn30_df[eod_vn30_df['date'] == itd_vn30f1m_df['date'].iloc[-1].normalize()]['close'].item()
open_interest = eod_vn30f1m_df[eod_vn30f1m_df['date'] == itd_vn30f1m_df['date'].iloc[-1].normalize()]['option'].item()
pivot_point = eod_vn30_df[eod_vn30_df['date'] == itd_vn30f1m_df['date'].iloc[-1].normalize()]['WPIVOT_P'].item()

itd_vn30f1m_df['pct_change'] = (itd_vn30f1m_df['close'] - vn30f1m_close_prev) / vn30f1m_close_prev
itd_vn30f1m_df['diff'] = itd_vn30f1m_df['close'] - vn30f1m_close_prev
itd_vn30f1m_df['basis'] = itd_vn30f1m_df['close'] - vn30_close_prev
itd_vn30f1m_df['OI'] = open_interest
itd_vn30f1m_df['pivot'] = pivot_point

- Viêt nhận xét ngắn

In [8]:
def daily_vn30f1m_overall_comment(df):
    prompt = f"""
    Đây là dữ liệu của chỉ số hợp đồng tương lai VN30F1M:
    {df.to_csv(index=False, sep='|', lineterminator='\n')}

    **Bối cảnh:**
    Hãy đóng vai là một nhà phân tích tài chính chuyên nghiệp và khách quan của một công ty chứng khoán. Nhiệm vụ của bạn là phân tích dữ liệu giá theo phút của hợp đồng tương lai (ticker VN30F1M) được cung cấp và viết một báo cáo ngắn gọn, súc tích về diễn biến của phiên giao dịch đó.

    **Dữ liệu đầu vào:**
    Dữ liệu là một chuỗi văn bản chứa thông tin giao dịch theo từng phút, bao gồm các cột: `date`, `ticker`, `open`, `high`, `low`, `close`.

    **Yêu cầu đầu ra:**
    Kết quả trả về phải tuân thủ nghiêm ngặt các quy tắc sau:

    * **Định dạng:** Chỉ trả về **duy nhất** đoạn văn phân tích. **Tuyệt đối không** thêm bất kỳ câu dẫn dắt, tiêu đề, hay lời chào nào (ví dụ: không được viết "Dưới đây là báo cáo..." hoặc "Sau khi phân tích...").
    * **Cấu trúc:** Đoạn văn phải có chính xác **3 câu**.
    * **Giọng điệu:** Chuyên nghiệp, khách quan, mang tính tường thuật, không đưa ra khuyến nghị mua/bán.
    * **Nguồn:** Mọi phân tích phải dựa hoàn toàn vào dữ liệu được cung cấp.
    * **LUÔN LUÔN** sử dụng ngày dạng "dd/mm" trong bài viết khi được yêu cầu.

    **Chi tiết hướng dẫn cho từng câu:**

    * **Câu 1: Bối cảnh và Diễn biến mở đầu (khoảng 15-20 từ)**
        * **Mục đích:** Giới thiệu bối cảnh phiên giao dịch và mô tả xu hướng ngay sau khi thị trường mở cửa.
        * **Hướng dẫn nội dung:**
            1.  Bắt đầu câu bằng cụm từ "Trong phiên giao dịch ngày `[DD/MM/YYYY]`...". Bạn phải tự động trích xuất và định dạng ngày từ cột `date` trong dữ liệu.
            2.  Nhận xét ngắn gọn về trạng thái mở cửa (ví dụ: "mở cửa khá ổn định", "mở cửa trong sắc xanh", "chịu áp lực ngay từ đầu").
            3.  Mô tả hành động giá ngay sau đó (ví dụ: "nhưng nhanh chóng chịu áp lực bán", "tuy nhiên đà tăng không duy trì được", "và tiếp tục nới rộng đà giảm").

    * **Câu 2: Diễn biến chính và Mức biến động (khoảng 15-20 từ)**
        * **Mục đích:** Tóm tắt xu hướng chủ đạo trong phiên và định lượng mức độ biến động mạnh nhất.
        * **Hướng dẫn nội dung:**
            1.  Mô tả xu hướng chính kéo dài trong phiên (ví dụ: "Xu hướng tiêu cực này chiếm ưu thế", "Lực mua chủ động xuyên suốt phiên").
            2.  Làm nổi bật mức biến động lớn nhất bằng cách tính chênh lệch giữa giá trị `high` cao nhất và `low` thấp nhất trong toàn bộ dữ liệu. Diễn đạt nó dưới dạng "đẩy chỉ số có thời điểm giảm sâu hơn `X` điểm" hoặc "giúp chỉ số có lúc tăng mạnh hơn `X` điểm".

    * **Câu 3: Tổng kết cuối phiên (khoảng 15-20 từ)**
        * **Mục đích:** Mô tả các nỗ lực vào cuối phiên và chốt lại trạng thái của giá đóng cửa.
        * **Hướng dẫn nội dung:**
            1.  Nhận xét về các diễn biến trong giai đoạn cuối phiên (ví dụ: "Mặc dù một nỗ lực phục hồi vào cuối phiên...", "Áp lực bán gia tăng trở lại khi kết phiên...").
            2.  Đưa ra kết luận về giá đóng cửa so với mặt bằng chung của phiên (ví dụ: "chỉ số vẫn đóng cửa ở vùng giá thấp", "bảo toàn được thành quả", "thu hẹp đáng kể đà giảm").
    """
    return prompt

overall_vn30f1m_comment = generate_content_with_model_dict(standard_model_dict, daily_vn30f1m_overall_comment(itd_vn30f1m_df), 'daily_vn30f1m_overall_comment')

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vn30f1m_overall_comment.


#### Lấy dữ liệu nntd VN30F1M

In [15]:
# Đọc tất cả các file FiinProX từ thư mục Downloads
fiinprox_files = glob.glob(os.path.join(str(Path.home() / "Downloads"), "*FiinProX*.xlsx"))

# Lọc lại các file chứa dữ liệu thị trường mở và có số cột > 6
td_finprox_files = []
for file in fiinprox_files:
    df = pd.read_excel(file)
    td_name_check = df.applymap(lambda x: isinstance(x, str) and "Thống kê thị trường_Tự doanh-Chi tiết" in x).any().any()
    if td_name_check:
        td_finprox_files.append(file)
if len(td_finprox_files) == 0:
    raise FileNotFoundError("❌ Chưa tải dữ liệu Tự Doanh từ FiinProX.")

# Sắp xếp fiinprox_files theo thời gian tạo (tăng dần) và lấy file mới nhất
last_td_fiinprox_file = sorted(fiinprox_files, key=os.path.getctime)[-1]

In [16]:
#Đọc dữ liệu Tự Doanh từ file mới nhất
td_vn30f1m_df = pd.read_excel(last_td_fiinprox_file, skiprows=8).iloc[:, :12]
td_vn30f1m_df = td_vn30f1m_df.dropna().iloc[:20].reset_index(drop=True)
td_vn30f1m_df = td_vn30f1m_df.drop(td_vn30f1m_df.columns[[3, 5, 7, 9]], axis=1)
td_vn30f1m_df.columns = ['ticker', 'date', 'buy_volume', 'sell_volume', 'buy_value', 'sell_value', 'net_value', 'net_volume']
td_vn30f1m_df[[col for col in td_vn30f1m_df.columns if '_value' in col]] = td_vn30f1m_df[[col for col in td_vn30f1m_df.columns if '_value' in col]]/1000
td_vn30f1m_df['type'] = 'TD'

#Kiểm tra xem dữ liệu Tự Doanh đã mới nhất chưa
td_date_check = td_vn30f1m_df['date'].max()
date_to_check = date_series.iloc[2].item()
if td_date_check <= date_to_check:
    raise ValueError(f"❌ Dữ liệu Tự Doanh không mới nhất. Ngày trong dữ liệu: {td_date_check}, ngày yêu cầu: {date_to_check}")
else:
    print(f"✅ Đã cập nhật dữ liệu Tự Doanh đã mới nhất. Ngày trong dữ liệu: {td_date_check.strftime('%d-%m-%Y')}")

#Lưu riêng ticker cho VN30F1M dùng sau này
vn30f1m_ticker = td_vn30f1m_df['ticker'].unique()[0]

#Dữ liệu NN từ Fireant
nn_vn30f1m_df = nntd_index_df[(nntd_index_df['ticker'] == 'VN30F1M') & (nntd_index_df['type'] == 'NN')].reset_index(drop=True)
nn_vn30f1m_df['ticker'] = vn30f1m_ticker

#Ghép bảng NN và TD
daily_8h45_nntd_df = pd.concat([nn_vn30f1m_df, td_vn30f1m_df], axis=0, ignore_index=True)

✅ Đã cập nhật dữ liệu Tự Doanh đã mới nhất. Ngày trong dữ liệu: 01-08-2025


#### Vẽ biểu đồ kĩ thuật

In [18]:
vn30f1m_chart_df = full_index_df[full_index_df['ticker'] == 'VN30F1M'].iloc[:120].sort_values('date').reset_index(drop=True)
vn30f1m_chart_df['ticker'] = vn30f1m_ticker
image_name = 'TA_DAILY_VN30F1M.png'

line_name_dict = {
    'SMA_20': 'SMA 20',
    'SMA_60': 'SMA 60',
    'week_open': 'WEEK OPEN',
    'month_open': 'MONTH OPEN',
    'quarter_open': 'QUARTER OPEN',
    'year_open': 'YEAR OPEN',
    'month_prev_high': 'LAST MHIGH',
    'month_prev_low': 'LAST MLOW',
    'quarter_prev_high': 'LAST QHIGH',
    'quarter_prev_low': 'LAST QLOW',
    'MFIBO_0382': 'MFIBO 0.382',
    'MFIBO_0500': 'MFIBO 0.500',
    'MFIBO_0618': 'MFIBO 0.618',
    'QFIBO_0382': 'QFIBO 0.382',
    'QFIBO_0500': 'QFIBO 0.500',
    'QFIBO_0618': 'QFIBO 0.618',
	'YFIBO_0382': 'YFIBO 0.382',
    'YFIBO_0500': 'YFIBO 0.500',
    'YFIBO_0618': 'YFIBO 0.618'
}

chart_config = create_chart_config(
	title_font_size=30,
	axis_font_size=24,
	tag_font_size=24,
	price_tag_font_size=24,
    min_spacing_ratio = 0.042,
	margin=dict(l=20, r=220, t=20, b=20, pad=10)
)

max_attempts = 5
for attempt in range(max_attempts):
    try:
        fig, fig_images = create_financial_chart(
            vn30f1m_chart_df, # DataFrame của bạn
            width=1400,
            height=1200,
            line_name_dict=line_name_dict,
            line_columns=list(line_name_dict.keys()),
            chart_config=chart_config,
            path='../output/cts_weekly_chart/',
            symbol_name='VN30F1M',
            image_name = image_name
        )
        upload_to_r2(fig_images, image_name)
        break
    except Exception as e:
        if attempt == max_attempts - 1:
            raise e
# fig.show()

In [42]:
def daily_vn30f1m_ta_analysis(df):
    prompt = f"""
Đây là dữ liệu của chỉ số hợp đồng tương lai VN30F1M:
    {df.to_csv(index=False, sep='|', lineterminator='\\n')}

**Vai trò:** Bạn là một **nhà phân tích kỹ thuật cấp cao**, chuyên về phái sinh, có nhiệm vụ đưa ra nhận định kỹ thuật khách quan và sắc bén.

**Nhiệm vụ:** Dựa trên dữ liệu và các chỉ báo kỹ thuật, hãy viết một báo cáo phân tích về trạng thái kỹ thuật của chỉ số cho **phiên kế tiếp**, tuân thủ nghiêm ngặt cấu trúc **8 câu, 3 đoạn**.

**Yêu cầu về nội dung, cấu trúc và định dạng:**
    Báo cáo phải gồm **8 câu, mỗi câu dài khoảng 20 đến 25 từ** và được chia thành **3 đoạn văn riêng biệt**. Bắt đầu thẳng vào nội dung, không có lời chào.

    **Đoạn 1 (3 câu): Phân tích Phiên Gần Nhất**
        * **Câu 1:** Nhận định tổng quan về diễn biến giá và xu hướng chính trong **phiên giao dịch gần nhất (ngày dd/mm)**.
        * **Câu 2:** Phân tích vai trò của hai đường **SMA 20 và SMA 60** trong việc xác nhận xu hướng ngắn hạn và trung hạn hiện tại.
        * **Câu 3:** Đánh giá chỉ báo **RSI 14**, nêu bật trạng thái hiện tại của chỉ báo (ví dụ: vùng trung tính, quá mua/bán) và ý nghĩa của nó.

    **Đoạn 2 (3 câu): Xác định các Vùng giá Cốt lõi**
        * **Câu 4:** Phân tích và xác định mốc hỗ trợ/kháng cự quan trọng theo **khung Tuần** (kết hợp O-H-L gần nhất và Fibonacci gần nhất).
        * **Câu 5:** Phân tích và xác định mốc hỗ trợ/kháng cự quan trọng theo **khung Tháng** (kết hợp O-H-L gần nhất và Fibonacci gần nhất).
        * **Câu 6:** Đưa ra kết luận và nhấn mạnh **một vùng hỗ trợ và một vùng kháng cự then chốt nhất** sẽ quyết định xu hướng trong các phiên tới.

    **Đoạn 3 (2 câu): Kết luận Kỹ thuật**
        * **Câu 7:** Đưa ra một câu nhận định tổng thể, khẳng định xu hướng kỹ thuật chủ đạo hiện tại của chỉ số (tăng, giảm hoặc đi ngang).
        * **Câu 8:** Ngay sau đó, chỉ ra các vùng giá hỗ trợ và kháng cự then chốt sẽ là tâm điểm quyết định cho các phiên giao dịch tiếp theo và giải thích tại sao.

**Yêu cầu về ngôn ngữ và trình bày:**
    * **Văn phong:** Sắc bén, khách quan, quyết đoán và đa dạng, tránh lặp lại cấu trúc câu.
    * **TUYỆT ĐỐI KHÔNG** sử dụng dấu ;.
    * **TUYỆT ĐỐI KHÔNG** được viết tên cột dữ liệu gốc và **KHÔNG** có lời chào hay câu dẫn dắt.
    * **TUYỆT ĐỐI KHÔNG** sử dụng các từ nối mang tính tổng kết ở đầu câu, cứ đi thẳng vào nội dung chính (ví dụ: "Tóm lại", "Nhìn chung", "Do đó", "Theo đó", "Tổng thể").
    * **LUÔN LUÔN** sử dụng ngày dạng "dd/mm" trong bài viết khi được yêu cầu.
    """
    return prompt

ta_vn30f1m_comment = generate_content_with_model_dict(standard_model_dict, daily_vn30f1m_ta_analysis(vn30f1m_chart_df), 'daily_vn30f1m_ta_analysis')


✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vn30f1m_ta_analysis.


#### Chiến lược mua bán

- Các vùng kháng cự và hỗ trợ gần nhất

In [ ]:
def get_nerest_levels(df_row: pd.DataFrame) -> pd.DataFrame:
    # Lấy giá đóng cửa từ hàng đầu tiên
    close_price = df_row['close'].iloc[0]

    # Danh sách các cột được coi là chỉ báo kỹ thuật để xét
    indicator_columns = [
        'SMA_20', 'SMA_60', 'week_open', 'month_open', 'quarter_open', 'year_open',
        'MFIBO_0382', 'MFIBO_0500', 'MFIBO_0618', 'QFIBO_0382', 'QFIBO_0500',
        'QFIBO_0618', 'YFIBO_0382', 'YFIBO_0500', 'YFIBO_0618', 'month_prev_high',
        'month_prev_low', 'quarter_prev_high', 'quarter_prev_low', 'WPIVOT_P'
    ]

    # Ánh xạ tên cột gốc sang tên ngắn gọn để hiển thị
    name_map = {
        "SMA_20": "Đường MA20",
        "SMA_60": "Đường MA60",
        "week_open": "Giá Open Tuần",
        "month_open": "Giá Open Thắng",
        "quarter_open": "Giá Open Quý",
        "year_open": "Giá Open Năm",
        "MFIBO_0382": "FIBO 0.382 Tháng",
        "MFIBO_0500": "FIBO 0.500 Tháng",
        "MFIBO_0618": "FIBO 0.618 Tháng",
        "QFIBO_0382": "FIBO 0.382 Quý",
        "QFIBO_0500": "FIBO 0.500 Quý",
        "QFIBO_0618": "FIBO 0.618 Quý",
        "YFIBO_0382": "FIBO 0.382 Năm",
        "YFIBO_0500": "FIBO 0.500 Năm",
        "YFIBO_0618": "FIBO 0.618 Năm",
        "month_prev_high": "Đỉnh tháng trước",
        "month_prev_low": "Đáy tháng trước",
        "quarter_prev_high": "Đỉnh quý trước",
        "quarter_prev_low": "Đáy quý trước",
        "WPIVOT_P": "W.Pivot"
    }

    levels = []
    for col in indicator_columns:
        # Kiểm tra xem cột có tồn tại trong DataFrame không
        if col in df_row.columns:
            value = df_row[col].iloc[0]
            if pd.notna(value):
                distance = abs(value - close_price)
                
                # Bỏ qua các giá trị bằng đúng giá đóng cửa
                if distance == 0:
                    continue

                if value > close_price:
                    level_type = 'Kháng cự'
                else:
                    level_type = 'Hỗ trợ'
                
                levels.append({
                    'type': level_type,
                    'name': name_map.get(col, col), # Lấy tên ngắn gọn
                    'value': value,
                    'distance': distance
                })

    # Tách ra 2 danh sách, sắp xếp theo khoảng cách và lấy 3 mục đầu tiên
    resistances = sorted([l for l in levels if l['type'] == 'Kháng cự'], key=lambda x: x['distance'])[:3]
    supports = sorted([l for l in levels if l['type'] == 'Hỗ trợ'], key=lambda x: x['distance'])[:3]

    # Kết hợp lại và tạo DataFrame kết quả
    top_levels_data = resistances + supports
    
    result_df = pd.DataFrame(top_levels_data)

    # Chỉ chọn các cột cần thiết cho kết quả cuối cùng
    return result_df[['type', 'name', 'value']]

# 2. Gọi hàm với DataFrame mẫu
daily_8h45_nearest_df = get_nerest_levels(vn30f1m_chart_df.iloc[[0]])

- Chi tiết chiến lược mua bán

In [93]:
def daily_vn30f1m_long_short(df, ta_vn30f1m_comment):
    prompt = f"""
    Đây là dữ liệu của chỉ số hợp đồng tương lai VN30F1M:
        {df.to_csv(index=False, sep='|', lineterminator='\\n')}

    Đây là nhận định từ chuyên gia về xu hướng thị trường:
        {ta_vn30f1m_comment}

**Bối cảnh và Vai trò**

Bạn là một nhà phânْتích kỹ thuật cao cấp tại một quỹ đầu tư, chịu trách nhiệm xây dựng các kịch bản giao dịch hàng ngày cho đội ngũ trader. Nhiệm vụ của bạn là chuyển hóa các phân tích thị trường phức tạp thành những kế hoạch hành động rõ ràng, định lượng và có thể thực thi ngay lập tức. Từ dữ liệu và nhận định được cung cấp, hãy soạn thảo một báo cáo chiến lược ngắn gọn cho ngày giao dịch tiếp theo.

**Mô tả Dữ liệu Đầu vào**

Bạn sẽ nhận được hai phần thông tin: 
1.  Một chuỗi dữ liệu thô chứa các thông số giá OHLCV (Open, High, Low, Close, Volume) và các chỉ báo kỹ thuật liên quan.
2.  Một đoạn văn nhận định của chuyên gia, tổng quan về xu hướng thị trường và xác định các ngưỡng hỗ trợ/kháng cự chính.

**Yêu cầu Nhiệm vụ**

Hãy viết một bản phân tích chiến lược giao dịch gồm 3 đoạn văn riêng biệt, độc lập, tuân thủ các yêu cầu chi tiết về cấu trúc và nội dung sau:

**Đoạn 1: Chiến lược Vị thế Long**
- **Bắt đầu đoạn văn ngay lập tức, không có câu giới thiệu.**
- **Câu 1:** Mô tả vùng vào lệnh đầu tiên, kết hợp **vùng giá, lý do kỹ thuật chi tiết, và mức Dừng Lỗ (Stop Loss) cụ thể** trong một câu văn duy nhất.
- **Câu 2:** Mô tả vùng vào lệnh thứ hai theo cấu trúc tương tự: **vùng giá, lý do kỹ thuật, và mức Dừng Lỗ.**
- **Câu 3 (kết luận):** Nêu hai mục tiêu Chốt Lời (Take Profit) chung. Ngay sau đó, tính toán và liệt kê lợi nhuận kỳ vọng (SỐ ĐIỂM) cho cả hai kịch bản vào lệnh (ví dụ: "...; nếu vào lệnh từ [vùng 1], lợi nhuận là X và Y điểm, còn từ [vùng 2], lợi nhuận là A và B điểm.").
- **LƯU Ý QUAN TRỌNG:**: Các vùng TP chỉ chọn tối đá 50 diểm từ giá hiện tại, các vùng CUTLOSS chọn tối đa 10 điểm từ giá hiện tại.
- Chỉ sử dụng thuật ngữ "Vị thế Long", tuyệt đối không dùng từ "mua".

**Đoạn 2: Chiến lược Vị thế Short**
- Xây dựng đoạn văn với cấu trúc y hệt Đoạn 1, nhưng tập trung vào kịch bản giá đảo chiều tại các vùng kháng cự.
- **Bắt đầu trực tiếp**, không giới thiệu.
- **Câu 1 & 2:** Mỗi câu mô tả một vùng vào lệnh, bao gồm **vùng giá, lý do kỹ thuật, và mức Dừng Lỗ** tương ứng.
- **Câu 3 (kết luận):** Nêu hai mục tiêu Chốt Lời chung và tính toán lợi nhuận kỳ vọng (SỐ ĐIỂM) cho cả hai kịch bản vào lệnh.
- **LƯU Ý QUAN TRỌNG:**: Các vùng TP chỉ chọn tối đá 50 diểm từ giá hiện tại, các vùng CUTLOSS chọn tối đa 10 điểm từ giá hiện tại.
- Chỉ sử dụng thuật ngữ "Vị thế Short", tuyệt đối không dùng từ "bán".

**Đoạn 3: Đánh giá và Ưu tiên**
- **Bắt đầu trực tiếp**, không giới thiệu.
- ** Câu 1: Hãy cho biết nên ưu tiên Vị thế Long hay Vị thế Short trong giai đoạn này.
- ** Câu 2: Nêu lý do súc tích cho sự ưu tiên đó và đưa ra một lưu ý quan trọng cho chiến lược còn lại.

**Yêu cầu chung về văn phong và định dạng:**
- Kêt quả đầu ra phải là tuân thủ nghiêm ngặt 3 đoạn văn như đã hướng dẫn, ko tự ý thêm cây hay đoạn hoặc xuống dòng nào khác.
- Kết thúc câu bằng dấu . và không sử dụng dấu ;.
- Sử dụng ngôn ngữ chuyên nghiệp, súc tích, khách quan.
- Không có các câu nối hay cụm từ giới thiệu giữa ba đoạn văn.
- Hạn chế tối đa việc lặp từ bằng cách sử dụng các từ đồng nghĩa và cấu trúc câu đa dạng.
    """
    return prompt

while True: # Vòng lặp sẽ chạy mãi mãi cho đến khi có lệnh 'break'
    prompt_instance = daily_vn30f1m_long_short(vn30f1m_chart_df, ta_vn30f1m_comment)
    raw_output = generate_content_with_model_dict(
        standard_model_dict, 
        prompt_instance, 
        'daily_vn30f1m_long_short'
    )
    paragraphs = [p.strip() for p in raw_output.strip().split('\n') if p.strip()]
    if len(paragraphs) == 3:
        long_short_vn30f1m_comment = raw_output
        break

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vn30f1m_long_short.


#### Lưu vào MSSQL

In [ ]:
daily_8h30_time_df = pd.DataFrame([{
    'time': time_series.iloc[0].item().strftime('%H:%M'),
    'date': date_series.iloc[0].item().strftime('%d/%m/%Y')
}])